In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np

import utils

In [ ]:
#### 2020 ####
column_semantics_2020 = {
    'Investment number': 'investment_id',
    'This investment belongs to project number…': 'project_id',
    'Commissioning Year': 'commissioning_year',
    'Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction': 'status',
    'Type of Element': 'asset_type',
    'Substation From': 'substation_1',
    'Substation To': 'substation_2',
    'Technology': 'ac_dc',
    'Total route length (km)': 'specified_length_km',
    'Description of the investment': 'description'
}

status_map_2020 = {
    1: 'under_consideration', # under consideration
    2: 'planned_not_yet_permitting', # planning, not permitting
    3: 'in_permitting', # in permitting
    4: 'under_construction'  # under construction
}
# No need to separate AC and DC as there is a separate column
# in the TYNDP data.
asset_type_map_2020 = {
    'ACTransmissionLine': 'line',
    'DCTransmissionLine': 'line',
    'OnshoreSubstation': 'substation',
    'OffshoreSubstation': 'substation',
    # 'cable' will be mapped to 'line' after adding column 'underground' of binary values.
    'OffshoreDCTransmissionCable': 'cable',
    'OffshoreACTransmissionCable': 'cable'
}

excel_2020, sheet_2020 = '2020/transmission.xlsx', 'Trans.Investments'

In [ ]:
#### 2018 ####
column_semantics_2018 = {
    'Investment ID': 'investment_id',
    'ExpectedCommissioningYear': 'commissioning_year',
    'Status': 'status',
    'ElementsType': 'asset_type',
    'From': 'substation_1',
    'To': 'substation_2',
    'TechnologyType': 'ac_dc',
    'VoltageLevel (kV)': 'voltage',
    'TotalRouteLength (km)': 'specified_length_km'
}

status_map_2018 = {
    'under consideration': 1,
    'planned but not yet permitting': 2,
    'permitting': 3,
    'under construction': 4
}
asset_type_map_2018 = {
    'Overhead Line': 'line',
    'Substation': 'substation',
    'Subsea Cable': 'cable',
    'Underground Cable': 'cable'
}

excel_2018, sheet_2018 =  r'2018/TYNDP_2018_Project_List.xlsx', 'Sheet1'

In [ ]:
#### 2016 ####
column_semantics_2016 = {
    'TYNDP 2016 Investment Index': 'investment_id',
    'TYNDP 2016 expected commissioning date': 'commissioning_year',
    'TYNDP 2016 status name': 'status',
    'Element type': 'asset_type',
    'From Substation name': 'substation_1',
    'To Substation name': 'substation_2',
    'AC or DC': 'ac_dc',
    'Voltage Level': 'voltage',
    'Line length [km]': 'specified_length_km'
}
# TODO: commissioned, planning, design
# TODO: rather specify rows we want to keep? E.g. via list
# TODO: map values to strings (semantics!), then choose only those that are 'in permitting' or 'under construction'
status_map_2016 = {
    'under consideration': 1,
    'Design & Permitting': 2,
    'permitting': 3,
    'under construction': 4
}
asset_type_map_2016 = {
    'Overhead Line': 'line',
    'Substation': 'substation',
    'Subsea Cable': 'cable',
    'Underground Cable': 'cable'
}

excel_2016, sheet_2016 = r'2016/TYNDP2016_Projects data.xlsx', 'Investments'

In [ ]:
# wanted = tyndp_to_pypsa.prepare_tyndp_data(excel_2010, sheet_2010, column_semantics_2010, status_map_2010, header_row=2)
# wanted = tyndp_to_pypsa.prepare_tyndp_data(excel_2012, sheet_2012, column_semantics_2012, status_map_2012, header_row=4)
#wanted = tyndp_to_pypsa.prepare_tyndp_data(excel_2014, sheet_2014, column_semantics_2014, status_map_2014)
tyndp = utils.prepare_tyndp_data(excel_2020, sheet_2020, column_semantics_2020, status_map_2020, asset_type_map=asset_type_map_2020,header_row=1, base_url=r'https://tyndp2020-project-platform.azurewebsites.net/projectsheets/transmission/')
tyndp.head()

In [ ]:
# tyndp.commissioning_year = utils.commissioning_dates_to_year(tyndp.commissioning_year)
tyndp['x1'], tyndp['y1'], tyndp['x2'], tyndp['y2'] = np.NAN, np.NAN, np.NAN, np.NAN

In [ ]:
tyndp.to_csv('2020/2020_curated.csv')

In [ ]:
# read in newest buses.csv
buses = (pd.read_csv('buses.csv', quotechar="'",
                     true_values='t', false_values='f',
                     dtype=dict(bus_id="str"))
         .set_index("bus_id")
         .drop(['station_id'], axis=1)
         .rename(columns=dict(voltage='v_nom')))
buses = buses.query('symbol == "Substation"')

In [ ]:
# TODO: maybe add option to plot pypsa-eur lines with switch (-> check whether line exists / is placed wrongly)

import plotly.express as px

fig = px.scatter_mapbox(buses, lat="y", lon="x", color_discrete_sequence=["fuchsia"], zoom=3, height=600)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [ ]:
# consecutive coordinates are interpreted as start and endpoints?
lats = [48, 50] 
lons = [10, 12]
fig = px.line_geo(lat=lats, lon=lons)# , color=3*['fuchsia'])
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()